<a href="https://colab.research.google.com/github/LaurePavard/plane-classification/blob/main/train_classification_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Récupération des données

In [1]:
!curl -O https://www.robots.ox.ac.uk/~vgg/data/fgvc-aircraft/archives/fgvc-aircraft-2013b.tar.gz
!tar zxf fgvc-aircraft-2013b.tar.gz
!mv fgvc-aircraft-2013b dataset

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2625M  100 2625M    0     0  7742k      0  0:05:47  0:05:47 --:--:-- 11.7M


## Traitement des données

Un fichier nous donne la variable y (le constructeur) et un autre fichier nous donnes les x (les images). Il nous faut un lien vers les deux fichiers pour construire notre modèle.

Problème de classification mutliclasse avec un grand nombre de classes (30 à 70 classes) 

In [2]:
import pathlib
#permet de gérer proprement les chemins d'accès

import pandas as pd

In [3]:
#création d'une variable de base 
#on écrit les variables en majuscules parce que pep 8 (convention de style pour coder en Python)
#pep : propositions d'amélioration de python
#pep 8 : une des recommandation est d'écrire les constantes du modèle en majuscule (initialisée au début mais pas modifiée par la suite)

#ici on indique le chemin où se trouvent les données/chemin d'accès aux données via DATA_DIR
DATA_DIR=pathlib.Path('dataset/data')

In [6]:
pd.read_csv(DATA_DIR / 'images_manufacturer_train.txt', sep=' ', names=['image_id','manufacturer'], usecols=['image_id','manufacturer'])
#le / fonctionne depuis n'importe quel système d'exploitation et concatène correctement le chemin d'accès
#séparateur = espace
#Attention, les noms composés sont aussi séparés par un espace.
#Pour compenser, on indique que le csv n'a que 2 colonnes et non 3 à l'aide de names
#usecols pour la version 1.4 

,image_id,manufacturer
0,1025794,Boeing
1,1340192,Boeing
2,56978,Boeing
3,698580,Boeing
4,450014,Boeing
...,...,...
3329,1577680,Yakovlev
3330,1826676,Yakovlev
3331,472681,Yakovlev
3332,1597829,Yakovlev


In [7]:
#Commenter le code pour bien comprendre comment il fonctionne et faciliter le travail sur le code dans la durée

In [ ]:
#L'id a enlevé les zero non significatifs alors que les fichiers ont les zéros devant, on va donc forcer Pandas a lire ces zéros en chaîne de caractère à l'aide de dtype
#les id sont maintenant des chaînes de caractère (on fait pareil pour les codes postaux)

In [10]:
pd.read_csv(DATA_DIR / 'images_manufacturer_train.txt', sep=' ', 
            names=['image_id','manufacturer'], 
            usecols=['image_id','manufacturer'], 
            dtype={'image_id':str},
            )

,image_id,manufacturer
0,1025794,Boeing
1,1340192,Boeing
2,0056978,Boeing
3,0698580,Boeing
4,0450014,Boeing
...,...,...
3329,1577680,Yakovlev
3330,1826676,Yakovlev
3331,0472681,Yakovlev
3332,1597829,Yakovlev


In [11]:
manufacturer_df=pd.read_csv(DATA_DIR / 'images_manufacturer_train.txt', sep=' ', 
            names=['image_id','manufacturer'], 
            usecols=['image_id','manufacturer'], 
            dtype={'image_id':str},
            )

In [ ]:
#bien vérifier les fichiers quand on les lit pour éviter les bugs à la fin

In [12]:
manufacturer_df['manufacturer'].value_counts(dropna=False)
#le dropna=False compte les valeurs manquantes
#Il n'y a pas de valeur manquante puisque na n'apparaît pas

Boeing         733
Airbus         434
Embraer        233
McDonnell      232
de             167
Canadair       134
Douglas        133
Cessna         133
British        133
Lockheed       102
Fokker         100
Dassault        67
Gulfstream      67
Beechcraft      67
Saab            67
Tupolev         66
ATR             66
Panavia         34
Antonov         34
Dornier         34
Yakovlev        34
Bombardier      33
Ilyushin        33
Fairchild       33
Piper           33
Cirrus          33
Eurofighter     33
Supermarine     33
Robin           33
Name: manufacturer, dtype: int64

In [13]:
manufacturer_df.isna().sum()

image_id        0
manufacturer    0
dtype: int64

In [ ]:
#Garantie qu'il n'y a pas de valeur manquante, s'il y en a le zéro est remplacé par 1

In [15]:
assert manufacturer_df['image_id'].isna().sum()==0, 'Valeur manquante dans image_id'
#Si c'est égal à zéro le assert va fonctionner, sinon il affiche "valeur manquante dans image_id"
#on peut faire plusieurs assert, si il ne se passe rien ça passe à l'étape d'après sinon il s'arrête

In [ ]:
manufacturer_df['manufacturer']=

###Aurait pu marcher mais non

In [16]:
manufacturer_df=pd.read_csv(DATA_DIR / 'images_manufacturer_train.txt', sep=' ', 
            names=['image_id','m1','m2'], 
            usecols=['image_id','m1','m2'], 
            dtype={'image_id':str},
            )
manufacturer_df['manufacturer']=manufacturer_df['m1']+' '+manufacturer_df['m2']
manufacturer_df['manufacturer'].unique()

#PAS LA BONNE METHODE

array([nan, 'British Aerospace', 'Lockheed Corporation',
       'Douglas Aircraft', 'McDonnell Douglas', 'de Havilland',
       'Lockheed Martin', 'Dassault Aviation', 'Bombardier Aerospace',
       'Gulfstream Aerospace', 'Cirrus Aircraft'], dtype=object)

###Méthode la plus astucieuse et qui fonctionne

In [30]:
manufacturer_df=pd.read_csv(DATA_DIR/ 'images_manufacturer_train.txt', sep='\t',
                            names=['all'],
                            dtype={'all':str},
                            )
manufacturer_df['image_id']=manufacturer_df['all'].apply(lambda x: x.split(' ')[0])
#la fonction split() découpe une chaîne de caractère

manufacturer_df['manufacturer']=manufacturer_df['all'].apply(lambda x: ' '.join(x.split(' ')[1:]))
#la fonction '<car>'.join(liste) concatène les éléments de liste en utilisant le séparateur <car>. (<car> pour caractère)

In [24]:
manufacturer_df

,all,image_id
0,1025794 Boeing,[Boeing]
1,1340192 Boeing,[Boeing]
2,0056978 Boeing,[Boeing]
3,0698580 Boeing,[Boeing]
4,0450014 Boeing,[Boeing]
...,...,...
3329,1577680 Yakovlev,[Yakovlev]
3330,1826676 Yakovlev,[Yakovlev]
3331,0472681 Yakovlev,[Yakovlev]
3332,1597829 Yakovlev,[Yakovlev]


In [31]:
manufacturer_df['manufacturer'].unique()

array(['Boeing', 'Airbus', 'ATR', 'Antonov', 'British Aerospace',
       'Beechcraft', 'Lockheed Corporation', 'Douglas Aircraft Company',
       'Canadair', 'Cessna', 'McDonnell Douglas', 'de Havilland', 'Robin',
       'Dornier', 'Embraer', 'Eurofighter', 'Lockheed Martin',
       'Dassault Aviation', 'Fokker', 'Bombardier Aerospace',
       'Gulfstream Aerospace', 'Ilyushin', 'Fairchild', 'Piper',
       'Cirrus Aircraft', 'Saab', 'Supermarine', 'Panavia', 'Tupolev',
       'Yakovlev'], dtype=object)

In [32]:
! grep ',' dataset/data/images_manufacturer_train.txt

In [33]:
! grep 'T' dataset/data/images_manufacturer_train.txt | head -3
#recherche le caractère T das le fichier et n'affiche que 3 lignes

0724121 ATR
0619697 ATR
2243949 ATR


In [35]:
! grep 'T' dataset/data/images_manufacturer_train.txt | wc -l
#wc: commpte le nombre d'éléments (-l: lignes, -c: caractères, -w: word)

132


In [37]:
! grep 'T' dataset/data/images_manufacturer_train.txt | wc
#Compte T dans les lignes, les caractères et les mots 

    132     264    1848


In [38]:
#Délimiteur, peut nous sauver en cas de fichier volumineux
#data science at the command line
!cut -f 1 -d ' ' dataset/data/images_manufacturer_train.txt | head 

1025794
1340192
0056978
0698580
0450014
1042824
0894380
1427680
0817494
0716386
